In [1]:
import pandas as pd
import numpy as np
import string
import re
import json

In [2]:
## Read the data into a dataframe

data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

In [3]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]
print(df.shape)

(189815, 6)


In [4]:
df = df[['headline', 'short_description']]

In [5]:
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

In [6]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations

    return text

In [7]:
df['clean_text']=df['input_data'].apply(clean_text)

In [8]:
dataset_news_category=list(df['clean_text'])

In [9]:
pattern = '[^a-z ]'
clr_x_data=[]
for sen in dataset_news_category:
    clr_x_data.append(re.sub(pattern, ' ', sen))

In [10]:
to_txt=[]
for sen in clr_x_data:
    to_txt.append(' '.join(sen.split()))

In [11]:
to_txt

['over million americans roll up sleeves for omicrontargeted covid boostershealth experts said it is too early to predict whether demand would match up with the million doses of the new boosters the us ordered for the fall',
 'american airlines flyer charged banned for life after punching flight attendant on videohe was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation according to the us attorneys office in los angeles',
 'of the funniest tweets about cats and dogs this week sept until you have a dog you dont understand what could be eaten',
 'the funniest tweets from parents this week sept accidentally put grownup toothpaste on my toddler s toothbrush and he screamed like i was cleaning his teeth with a carolina reaper dipped in tabasco sauce',
 'woman who called cops on black birdwatcher loses lawsuit against exemployeramy cooper accused investment firm franklin templeton of unfairly firing her and branding her a racist after video of th

In [12]:
import pickle

with open("nc_text.pkl","wb") as f:
    pickle.dump(to_txt, f)

In [12]:
len(to_txt)

189815

In [13]:
#문장 끝에 온점 추가
for i in range(len(to_txt)):
    to_txt[i]=to_txt[i] + " ."

In [14]:
import numpy as np

train_data=np.array(to_txt)

In [15]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
for i in range(0,train_data.shape[0],6):
    text = "\n".join(train_data[i:i+6].tolist())
    fp = open("swb/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

In [16]:
#소문자
from collections import Counter

texts = " ".join(train_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w")
fp.write(text)
fp.close()

Number of tokens in Training data =  5895932
Size of Vocab 230445


In [21]:
#하이퍼 파라미터 옵션(학습이 끝난 뒤엔 꼭 n_character=262로 해줘야 하므로 재설정. 기본값이 261로 돼있을 것임)
options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 262,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 1024, #엘모 차원 256으로 하기위한 옵션: 1024/엘모 차원 1024로 하기 위한 옵션: 4096 
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 128,   #엘모 차원 256으로 하기위한 옵션: 128/엘모 차원 1024로 하기 위한 옵션: 512
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': 5895932,   #위 코드에서 찾은 Number of tokens in Training data
     'batch_size': 64,
     'n_tokens_vocab': 230445,    #위 코드에서 찾은 Size of Vocab
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [22]:
import json
file_path = "./swb/checkpoint/options.json"
with open(file_path, 'w') as outfile:
    json.dump(options, outfile, indent=4)